# Composability

This notebook shows off some basic functionality around LangChain Expression Language, which makes it really easy to compose arbitrary chains.

For a much deeper dive, see:

- [Full LangChain Expression Language Documentation](https://python.langchain.com/docs/expression_language/)

## Basic Composability

In [7]:
import logging
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [8]:
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

load_dotenv()

True

In [9]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

In [10]:
chain = prompt | model | output_parser

In [11]:
chain.invoke({"topic": "bears"})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


"Sure, here's a bear joke for you: \n\nWhy don't bears wear shoes?\n\nBecause they have bear feet!"

### Batching

In [12]:
chain.batch([{"topic": "bear"}, {"topic": "clowns"}])

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


["Why don't bears like fast food?\n\nBecause they can't catch it!",
 'Why don\'t clowns like talking on the phone? \n\nBecause they\'re afraid of getting a "call" from their shoe!']

### Streaming

In [13]:
for s in chain.stream({"topic": "bears"}):
    print(s)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Sure
,
 here
's
 a
 bear
-related
 joke
 for
 you
:


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!


## RunnablePassthrough

In [14]:
from langchain_community.retrievers.tavily_search_api import TavilySearchAPIRetriever

retriever= TavilySearchAPIRetriever()

prompt = ChatPromptTemplate.from_template("""Answer the question based only on the context provided:

Context: {context}

Question: {question}""")

In [15]:
chain = prompt | model | output_parser

In [16]:
question = "what is langsmith"
context = "langsmith is a testing and observability platform built by the langchain team"
chain.invoke({"question": question, "context": context})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Langsmith is a testing and observability platform developed by the Langchain team.'

In [17]:
from langchain_core.runnables import RunnablePassthrough

retrieval_chain = RunnablePassthrough.assign(
    context=(lambda x: x["question"]) | retriever
) | chain

In [18]:
retrieval_chain.invoke({"question": "what is langsmith"})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'LangSmith is a platform that helps trace, evaluate, and monitor language model applications and intelligent agents. It provides tools for debugging, testing, and evaluating chains and agents built on any Language Model (LLM) framework. LangSmith seamlessly integrates with LangChain, an open-source framework for building with LLMs.'

## RunnableParallel

In [19]:
from langchain_core.runnables import RunnableParallel

In [20]:
prompt = ChatPromptTemplate.from_template("""{question}""")
simple_chain = prompt | model | output_parser

In [21]:
parallel_chain = RunnableParallel({
    "retrieved_answer": retrieval_chain,
    "simple_answer": simple_chain
})

In [22]:
parallel_chain.invoke({"question": "what is langsmith"})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'retrieved_answer': 'LangSmith is a platform that helps trace and evaluate language model applications and intelligent agents, allowing developers to move from prototyping to production. It provides features such as debugging, testing, evaluating, and monitoring chains and intelligent agents built on any Language Model (LLM) framework. LangSmith seamlessly integrates with LangChain, an open-source framework for building with LLMs.',
 'simple_answer': 'I\'m sorry, but I couldn\'t find any specific information about "Langsmith." It is possible that it could be a surname or a company name, but without further context, it is difficult to provide a more accurate answer.'}

In [23]:
for s in parallel_chain.stream({"question": "what is langsmith"}):
    print(s)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'simple_answer': ''}
{'simple_answer': 'I'}
{'simple_answer': "'m"}
{'simple_answer': ' sorry'}
{'simple_answer': ','}
{'simple_answer': ' but'}
{'simple_answer': ' I'}
{'simple_answer': ' couldn'}
{'simple_answer': "'t"}
{'simple_answer': ' find'}
{'simple_answer': ' any'}
{'simple_answer': ' information'}
{'simple_answer': ' on'}
{'simple_answer': ' "'}
{'simple_answer': 'lang'}
{'simple_answer': 'smith'}
{'simple_answer': '."'}
{'simple_answer': ' It'}
{'simple_answer': ' could'}
{'simple_answer': ' potentially'}
{'simple_answer': ' be'}
{'simple_answer': ' a'}
{'simple_answer': ' term'}
{'simple_answer': ' or'}
{'simple_answer': ' a'}
{'simple_answer': ' name'}
{'simple_answer': ' that'}
{'simple_answer': ' is'}
{'simple_answer': ' not'}
{'simple_answer': ' widely'}
{'simple_answer': ' recognized'}
{'simple_answer': ' or'}
{'simple_answer': ' associated'}
{'simple_answer': ' with'}
{'simple_answer': ' a'}
{'simple_answer': ' specific'}
{'simple_answer': ' meaning'}
{'simple_answer

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'retrieved_answer': ''}
{'retrieved_answer': 'Lang'}
{'retrieved_answer': 'Smith'}
{'retrieved_answer': ' is'}
{'retrieved_answer': ' a'}
{'retrieved_answer': ' platform'}
{'retrieved_answer': ' that'}
{'retrieved_answer': ' helps'}
{'retrieved_answer': ' trace'}
{'retrieved_answer': ' and'}
{'retrieved_answer': ' evaluate'}
{'retrieved_answer': ' language'}
{'retrieved_answer': ' model'}
{'retrieved_answer': ' applications'}
{'retrieved_answer': ' and'}
{'retrieved_answer': ' intelligent'}
{'retrieved_answer': ' agents'}
{'retrieved_answer': '.'}
{'retrieved_answer': ' It'}
{'retrieved_answer': ' allows'}
{'retrieved_answer': ' users'}
{'retrieved_answer': ' to'}
{'retrieved_answer': ' move'}
{'retrieved_answer': ' from'}
{'retrieved_answer': ' prototype'}
{'retrieved_answer': ' to'}
{'retrieved_answer': ' production'}
{'retrieved_answer': ' and'}
{'retrieved_answer': ' provides'}
{'retrieved_answer': ' documentation'}
{'retrieved_answer': ' for'}
{'retrieved_answer': ' reference'}
{

In [24]:
result = {}
for s in parallel_chain.stream({"question": "what is langsmith"}):
    for k,v in s.items():
        if k not in result:
            result[k] = ""
        result[k] += v
    print(result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'simple_answer': ''}
{'simple_answer': 'I'}
{'simple_answer': 'I couldn'}
{'simple_answer': "I couldn't"}
{'simple_answer': "I couldn't find"}
{'simple_answer': "I couldn't find any"}
{'simple_answer': "I couldn't find any specific"}
{'simple_answer': "I couldn't find any specific information"}
{'simple_answer': "I couldn't find any specific information about"}
{'simple_answer': 'I couldn\'t find any specific information about "'}
{'simple_answer': 'I couldn\'t find any specific information about "Lang'}
{'simple_answer': 'I couldn\'t find any specific information about "Langsmith'}
{'simple_answer': 'I couldn\'t find any specific information about "Langsmith."'}
{'simple_answer': 'I couldn\'t find any specific information about "Langsmith." It'}
{'simple_answer': 'I couldn\'t find any specific information about "Langsmith." It could'}
{'simple_answer': 'I couldn\'t find any specific information about "Langsmith." It could be'}
{'simple_answer': 'I couldn\'t find any specific informat

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'simple_answer': 'I couldn\'t find any specific information about "Langsmith." It could be a misspelling or a lesser-known term or name. If you have any additional context or information, I might be able to assist you further.', 'retrieved_answer': ''}
{'simple_answer': 'I couldn\'t find any specific information about "Langsmith." It could be a misspelling or a lesser-known term or name. If you have any additional context or information, I might be able to assist you further.', 'retrieved_answer': 'Lang'}
{'simple_answer': 'I couldn\'t find any specific information about "Langsmith." It could be a misspelling or a lesser-known term or name. If you have any additional context or information, I might be able to assist you further.', 'retrieved_answer': 'LangSmith'}
{'simple_answer': 'I couldn\'t find any specific information about "Langsmith." It could be a misspelling or a lesser-known term or name. If you have any additional context or information, I might be able to assist you furthe